# 🏇 NAR 全レース取得
以下の設定変数を変更して実行してください。NAR（地方競馬）のデータを日付順に取得します。

In [ ]:
# Google Driveをマウントする場合のみ実行してください
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import re
import time
from datetime import datetime

class RaceScraper:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def _get_soup(self, url):
        try:
            time.sleep(1) # Be polite
            response = requests.get(url, headers=self.headers, timeout=10)
            response.encoding = response.apparent_encoding
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Error fetching {url}: {e}")
        return None

    def get_past_races(self, horse_id, n_samples=5):
        """
        Fetches past n_samples race results for a given horse_id from netkeiba db.
        Returns a DataFrame of past races.
        """
        url = f"https://db.netkeiba.com/horse/result/{horse_id}/"
        soup = self._get_soup(url)
        if not soup:
            return pd.DataFrame()

        # The results are usually in a table with class "db_h_race_results"
        table = soup.select_one("table.db_h_race_results")
        if not table:
            # Try to find any table with "着順"
            tables = soup.find_all("table")
            for t in tables:
                if "着順" in t.text:
                    table = t
                    break
        
        if not table:
            return pd.DataFrame()

        # Parse table
        # We need to manually parse to get clean data and handle links if needed (though for past data, text is mostly fine)
        # pd.read_html is easier for the table
        try:
            df = pd.read_html(io.StringIO(str(table)))[0]
            
            # Basic cleaning
            df = df.dropna(how='all')
            
            # The columns in db.netkeiba are roughly:
            # 日付, 開催, 天気, R, レース名, 映像, 頭数, 枠番, ... 着順, ... 通過, ...
            
            # We want to keep: Date, Race Name, Course info, Rank, Time, Passing (Style)
            
            # Normalize column names (remove spaces/newlines)
            df.columns = df.columns.astype(str).str.replace(r'\s+', '', regex=True)

            # Filter rows that look like actual races (Date column exists)
            if '日付' in df.columns:
                df['date_obj'] = pd.to_datetime(df['日付'], format='%Y/%m/%d', errors='coerce')
                df = df.dropna(subset=['date_obj'])
                df = df.sort_values('date_obj', ascending=False)
                
            # Take top N
            if n_samples:
                df = df.head(n_samples)
            
            # Process Run Style (Leg Type)
            if '通過' in df.columns:
                df['run_style_val'] = df['通過'].apply(self.extract_run_style)
            else:
                df['run_style_val'] = 3 # Unknown

            # Extract/Rename Columns
            # We want: 日付, 開催, 天気, R, レース名, 映像, 頭数, 枠番, ... 着順, ... 通過, ...
            # Important: '上り' (3F), '馬体重', '騎手'
            
            # Map standard columns if they exist
            column_map = {
                '日付': 'date',
                '開催': 'venue',
                '天気': 'weather',
                'レース名': 'race_name',
                '着順': 'rank',
                '枠番': 'waku',
                '馬番': 'umaban',
                '騎手': 'jockey',
                '斤量': 'weight_carried',
                '馬場': 'condition', # 良/重/稍重 etc.
                'タイム': 'time',
                '着差': 'margin',
                '上り': 'last_3f',
                '通過': 'passing',
                '馬体重': 'horse_weight',
                'run_style_val': 'run_style',
                '単勝': 'odds',
                'オッズ': 'odds',
                '距離': 'raw_distance' # e.g. "芝1600"
            }
            
            # Rename available columns
            df.rename(columns=column_map, inplace=True)
            
            # Extract Surface and Distance from 'raw_distance'
            if 'raw_distance' in df.columns:
                def parse_dist(x):
                    if not isinstance(x, str): return None, None
                    # "芝1600", "ダ1200", "障3000"
                    # Sometimes "芝1600" or just "1600"
                    surf = None
                    dist = None
                    if '芝' in x: surf = '芝'
                    elif 'ダ' in x: surf = 'ダ'
                    elif '障' in x: surf = '障'
                    
                    # Extract number
                    match = re.search(r'(\d+)', x)
                    if match:
                        dist = int(match.group(1))
                    return surf, dist

                parsed = df['raw_distance'].apply(parse_dist)
                df['course_type'] = parsed.apply(lambda x: x[0])
                df['distance'] = parsed.apply(lambda x: x[1])
            else:
                df['course_type'] = None
                df['distance'] = None

            # Coerce numeric
            if 'rank' in df.columns:
                df['rank'] = pd.to_numeric(df['rank'], errors='coerce')
            
            if 'odds' in df.columns:
                 df['odds'] = pd.to_numeric(df['odds'], errors='coerce')
            
            # Fill missing
            for target_col in list(column_map.values()) + ['course_type', 'distance']:
                if target_col not in df.columns:
                    df[target_col] = None
                
            return df
            
        except Exception as e:
            print(f"Error parsing past races for {horse_id}: {e}")
            return pd.DataFrame()

    def extract_run_style(self, passing_str):
        """
        Converts passing order string (e.g., "1-1-1", "10-10-12") to run style (1,2,3,4).
        1: Nige (Escape) - Lead at 1st corner
        2: Senkou (Leader) - Within first ~4 or so
        3: Sashi (Mid) - Midpack
        4: Oikomi (Chaser) - Back
        Returns integer code.
        """
        if not isinstance(passing_str, str):
            return 3 # Default to Mid
            
        # Clean string "1-1-1" -> [1, 1, 1]
        try:
            cleaned = re.sub(r'[^0-9-]', '', passing_str)
            parts = [int(p) for p in cleaned.split('-') if p]
            
            if not parts:
                return 3
                
            first_corner = parts[0]
            
            # Heuristics
            if first_corner == 1:
                return 1 # Nige
            elif first_corner <= 4:
                return 2 # Senkou
            elif first_corner <= 9: # Assuming standard field size of 10-16, 9 is mid-ish limit? 
                # Actually "Sashi" is usually mid-rear. 
                # Let's say: 1=Lead, 2-4=Front, 5-10=Mid, >10=Back
                return 3 # Sashi
            else:
                return 4 # Oikomi
                
        except:
            return 3

    def scrape_race_with_history(self, race_id):
        """
        Detailed scraper that enters a race_result page, finding horse IDs, 
        then fetches history for each horse.
        Returns a dictionary or structured object with the race result + history.
        """
        url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
        soup = self._get_soup(url)
        if not soup:
            return None
            
        # 1. Parse Main Result Table to get Horse IDs and basic result
        # Note: auto_scraper already does some of this, but we need Horse IDs specifically.
        # "All_Result_Table"
        
        result_data = []
        
        table = soup.find("table", id="All_Result_Table")
        if not table:
            return None
            
        rows = table.find_all("tr", class_="HorseList")
        
        print(f"Found {len(rows)} horses in race {race_id}. Fetching histories...")
        
        for row in rows:
            # Extract basic info
            rank_elem = row.select_one(".Rank")
            rank = rank_elem.text.strip() if rank_elem else ""
            
            horse_name_elem = row.select_one(".Horse_Name a")
            horse_name = horse_name_elem.text.strip() if horse_name_elem else ""
            horse_url = horse_name_elem.get("href") if horse_name_elem else ""
            
            # Extract ID from URL
            # https://db.netkeiba.com/horse/2018105027
            horse_id = None
            if horse_url:
                match = re.search(r'/horse/(\d+)', horse_url)
                if match:
                    horse_id = match.group(1)
            
            if not horse_id:
                print(f"  Skipping {horse_name} (No ID)")
                continue

            print(f"  Fetching history for {horse_name} ({horse_id})...")
            
            # 2. Get Past History
            df_past = self.get_past_races(horse_id, n_samples=5)
            
            # 3. Structure Data
            # converting df_past to a list of dicts or flattened fields
            history = []
            if not df_past.empty:
                for idx, r in df_past.iterrows():
                    # Extract relevant columns
                    # We need at least: Rank, RunStyle, Time(Seconds?), Pace?
                    # For now just dump raw-ish data
                    hist_item = {
                        "date": r.get('日付'),
                        "race_name": r.get('レース名'),
                        "rank": r.get('着順'),
                        "passing": r.get('通過'),
                        "run_style": r.get('run_style_val'),
                        "time": r.get('タイム'),
                        # Add more as needed for Feature Engineering
                    }
                    history.append(hist_item)
            
            entry = {
                "race_id": race_id,
                "horse_id": horse_id,
                "horse_name": horse_name,
                "rank": rank,
                "history": history
            }
            result_data.append(entry)
            
        return result_data

    def get_horse_profile(self, horse_id):
        """
        Fetches horse profile to get pedigree (Father, Mother, Grandfather(BMS)).
        Returns a dictionary or None.
        """
        # Use pedigree page for reliable bloodline data
        url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
        soup = self._get_soup(url)
        if not soup:
            return None
        
        # Parse Blood Table
        # table class="blood_table"
        
        data = {
            "father": "",
            "mother": "",
            "bms": ""
        }
        
        try:
            table = soup.select_one("table.blood_table")
            if table:
                rows = table.find_all("tr")
                # 5-generation table has 32 rows usually
                # Father at Row 0 (rowspan 16)
                # Mother at Row 16 (rowspan 16)
                
                if len(rows) >= 17:
                    # Father: Row 0, Col 0
                    r0 = rows[0].find_all("td")
                    if r0:
                        txt = r0[0].text.strip()
                        # Clean: "スクリーンヒーロー\n2004 栗毛..." -> "スクリーンヒーロー"
                        # Take first line
                        data["father"] = txt.split('\n')[0].strip()
                        
                    # Mother & BMS: Row 16
                    r16 = rows[16].find_all("td")
                    if len(r16) >= 2:
                        # Mother
                        m_txt = r16[0].text.strip()
                        data["mother"] = m_txt.split('\n')[0].strip()
                        
                        # BMS (Mother's Father)
                        bms_txt = r16[1].text.strip()
                        data["bms"] = bms_txt.split('\n')[0].strip()
                        
        except Exception as e:
            print(f"Error parsing profile for {horse_id}: {e}")
            
        return data

    def get_race_metadata(self, race_id):
        """
        Fetches metadata for a specific race ID from Netkeiba.
        Returns dict with: race_name, date, venue, course_type, distance, weather, condition, turn
        """
        url = f"https://race.netkeiba.com/race/result.html?race_id={race_id}"
        soup = self._get_soup(url)
        if not soup:
            return None
            
        data = {
            "race_name": "",
            "date": "",
            "venue": "",
            "course_type": "",
            "distance": "",
            "weather": "",
            "condition": "",
            "turn": "", # New: Dictionary key for turn direction
            "race_id": race_id
        }
        
        try:
            # Race Name
            title_elem = soup.select_one(".RaceName")
            if title_elem:
                data["race_name"] = title_elem.text.strip()
                
            # Date & Venue & Conditions
            # <div class="RaceData01">... 2023年1月5日 ... 1回中山1日 ...</div>
            # Content: "15:35発走 / 芝1600m (右 外) / 天候:晴 / 馬場:良"
            
            rd1 = soup.select_one(".RaceData01")
            
            if rd1:
                txt = rd1.text.strip()
                
                # Weather
                if "天候:晴" in txt: data["weather"] = "晴"
                elif "天候:曇" in txt: data["weather"] = "曇"
                elif "天候:小雨" in txt: data["weather"] = "小雨"
                elif "天候:雨" in txt: data["weather"] = "雨"
                elif "天候:雪" in txt: data["weather"] = "雪"
                
                # Condition
                if "馬場:良" in txt: data["condition"] = "良"
                elif "馬場:稍" in txt: data["condition"] = "稍重" # Covers 稍重
                elif "馬場:重" in txt: data["condition"] = "重"
                elif "馬場:不良" in txt: data["condition"] = "不良"
                
                # Course & Distance ("芝1600m")
                # Regex for "芝", "ダ", "障" followed by digits
                match = re.search(r'(芝|ダ|障)(\d+)m', txt)
                if match:
                    ctype_raw = match.group(1)
                    if ctype_raw == "芝": data["course_type"] = "芝"
                    elif ctype_raw == "ダ": data["course_type"] = "ダート"
                    elif ctype_raw == "障": data["course_type"] = "障害"
                    
                    data["distance"] = match.group(2)
                
                # Turn Direction ("右", "左", "直線")
                # Usually in parentheses like "(右)" or "(左)" or "(芝 左)"
                if "右" in txt: data["turn"] = "右"
                elif "左" in txt: data["turn"] = "左"
                elif "直線" in txt: data["turn"] = "直"

            # Date
            # Try finding date in Title or dedicated element
            date_elem = soup.select_one("dl#RaceList_DateList dd.Active") 
            if date_elem:
                 # Usually "1月5日(金)" - needs Year
                 # We can rely on the fact that race_id contains year (2025...)
                 # But let's look for YYYY年 in the whole text or title
                 pass
            
            # Fallback Date from Title Tag or Meta
            if not data["date"]:
                 meta_title = soup.title.text if soup.title else ""
                 match_date = re.search(r'(\d{4}年\d{1,2}月\d{1,2}日)', meta_title)
                 if match_date:
                     data["date"] = match_date.group(1)

        except Exception as e:
            print(f"Error parsing metadata for {race_id}: {e}")
            
        return data

if __name__ == "__main__":
    # Test
    scraper = RaceScraper()
    print("Running test...")
    # Example: Do Deuce (2019105219)
    # url = "https://db.netkeiba.com/horse/2019105219/"
    # print(f"Fetching {url}")
    df = scraper.get_past_races("2019105219")
    if df.empty:
        print("DF is empty. Checking raw soup for 'db_h_race_results'...")
        soup = scraper._get_soup(f"https://db.netkeiba.com/horse/result/2019105219/")
        if soup:
             t = soup.select_one("table.db_h_race_results")
             print(f"Selector 'table.db_h_race_results' found: {t is not None}")
             if not t:
                 print("Trying fallback 'table' with '着順'...")
                 tables = soup.find_all("table")
                 found = False
                 for i, tbl in enumerate(tables):
                     print(f"Table {i} classes: {tbl.get('class')}")
                     if "着順" in tbl.text or "着 順" in tbl.text or "日付" in tbl.text:
                         print("Found a table with '着順/日付'.")
                         # print(str(tbl)[:200])
                         t = tbl
                         found = True
                         break
                 if not found:
                     print("No table with '着順' found in soup.")
                     print("Soup snippet:", soup.text[:500])
                 else:
                    # Retry parsing with found table
                     try:
                        df = pd.read_html(str(t))[0]
                        print("Retry DF Head:")
                        print(df.head())
                     except Exception as e:
                        print(f"Retry parsing failed: {e}")
        else:
            print("Soup is None.")
    else:
        print(df.head())
        print("Columns:", df.columns)


In [ ]:

# 2026-01-08: Rich Scraping Logic (JRA/NAR Universal)
# This code handles scraping of Race Result + Horse History + Pedigree in one pass.

import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
import re
from datetime import datetime
import urllib.parse
import time
import random
from tqdm.auto import tqdm

# --- RaceScraper Helper Class (Embedded) ---
class RaceScraper:
    def __init__(self):
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def _get_soup(self, url):
        try:
            time.sleep(1) # Be polite
            response = requests.get(url, headers=self.headers, timeout=10)
            response.encoding = response.apparent_encoding
            if response.status_code == 200:
                return BeautifulSoup(response.text, 'html.parser')
        except Exception as e:
            print(f"Error fetching {url}: {e}")
        return None

    def get_horse_profile(self, horse_id):
        """
        Fetches horse profile to get pedigree (Father, Mother, Grandfather(BMS)).
        Returns a dictionary or None.
        """
        url = f"https://db.netkeiba.com/horse/ped/{horse_id}/"
        soup = self._get_soup(url)
        data = {"father": "", "mother": "", "bms": ""}
        if not soup: return data

        try:
            table = soup.select_one("table.blood_table")
            if table:
                rows = table.find_all("tr")
                if len(rows) >= 17:
                    # Father: Row 0
                    r0 = rows[0].find_all("td")
                    if r0:
                        txt = r0[0].text.strip()
                        data["father"] = txt.split('\n')[0].strip()

                    # Mother & BMS: Row 16
                    r16 = rows[16].find_all("td")
                    if len(r16) >= 2:
                        m_txt = r16[0].text.strip()
                        data["mother"] = m_txt.split('\n')[0].strip()
                        bms_txt = r16[1].text.strip()
                        data["bms"] = bms_txt.split('\n')[0].strip()
        except Exception as e:
            pass # Silent fail for profile
        return data

    def extract_run_style(self, passing_str):
        if not isinstance(passing_str, str): return 3
        try:
            cleaned = re.sub(r'[^0-9-]', '', passing_str)
            parts = [int(p) for p in cleaned.split('-') if p]
            if not parts: return 3
            first_corner = parts[0]
            if first_corner == 1: return 1 # Nige
            elif first_corner <= 4: return 2 # Senkou
            elif first_corner <= 9: return 3 # Sashi
            else: return 4 # Oikomi
        except: return 3

    def get_past_races(self, horse_id, current_race_date, n_samples=5):
        """
        Fetches past n_samples race results.
        Filters out races AFTER current_race_date.
        Returns a DataFrame.
        """
        url = f"https://db.netkeiba.com/horse/result/{horse_id}/"
        soup = self._get_soup(url)
        if not soup: return pd.DataFrame()

        table = soup.select_one("table.db_h_race_results")
        if not table:
             tables = soup.find_all("table")
             for t in tables:
                 if "着順" in t.text:
                     table = t
                     break
        if not table: return pd.DataFrame()

        try:
            df = pd.read_html(io.StringIO(str(table)))[0]
            df = df.dropna(how='all')
            # Normalize columns
            df.columns = df.columns.astype(str).str.replace(r'\s+', '', regex=True)
            
            # Date Parsing
            if '日付' in df.columns:
                df['date_obj'] = pd.to_datetime(df['日付'], format='%Y/%m/%d', errors='coerce')
                df = df.dropna(subset=['date_obj'])
                
                # Filter past races only
                if isinstance(current_race_date, str):
                    current_date = pd.to_datetime(current_race_date)
                else:
                    current_date = pd.to_datetime(current_race_date)
                    
                df = df[df['date_obj'] < current_date]
                df = df.sort_values('date_obj', ascending=False)
            
            if df.empty: return df

            # Limit to N
            df = df.head(n_samples)

            # Extract Run Style
            if '通過' in df.columns:
                df['run_style_val'] = df['通過'].apply(self.extract_run_style)
            else:
                df['run_style_val'] = 3

            # Column Mapping
            column_map = {
                '日付': 'date', '開催': 'venue', '天気': 'weather', 'レース名': 'race_name',
                '着順': 'rank', '枠番': 'waku', '馬番': 'umaban', '騎手': 'jockey',
                '斤量': 'weight_carried', '馬場': 'condition', 'タイム': 'time',
                '着差': 'margin', '上り': 'last_3f', '通過': 'passing', '馬体重': 'horse_weight',
                'run_style_val': 'run_style', '単勝': 'odds', 'オッズ': 'odds', '距離': 'raw_distance'
            }
            df.rename(columns=column_map, inplace=True)
            
            # Parse Distance/Course
            if 'raw_distance' in df.columns:
                def parse_dist(x):
                    if not isinstance(x, str): return None, None
                    surf = None; dist = None
                    if '芝' in x: surf = '芝'
                    elif 'ダ' in x: surf = 'ダ'
                    elif '障' in x: surf = '障'
                    match = re.search(r'(\d+)', x)
                    if match: dist = int(match.group(1))
                    return surf, dist
                
                parsed = df['raw_distance'].apply(parse_dist)
                df['course_type'] = parsed.apply(lambda x: x[0])
                df['distance'] = parsed.apply(lambda x: x[1])
            else:
                df['course_type'] = None; df['distance'] = None

            return df
        except Exception as e:
            return pd.DataFrame()

# --- Main Rich Scraper Function ---
def scrape_race_rich(url, existing_race_ids=None, max_retries=3):
    """
    Scrapes Race + History + Pedigree in one go.
    """
    print(f"  レース解析中: {url}")
    headers = {"User-Agent": "Mozilla/5.0"}
    
    # 1. Fetch Race Page
    try:
        resp = requests.get(url, headers=headers, timeout=15)
        resp.encoding = 'EUC-JP'
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        # --- Metadata (Date, Venue, Race Name, etc) ---
        # (This part reuses logic from original scrape_jra_race)
        h1_elem = soup.select_one("div.header_line h1 .txt")
        full_text = h1_elem.text.strip() if h1_elem else (soup.h1.text.strip() if soup.h1 else "")
        
        date_text = ""; venue_text = ""; kai = "01"; day = "01"; r_num = "10"
        match_date = re.search(r'(\d{4}年\d{1,2}月\d{1,2}日)', full_text)
        if match_date: date_text = match_date.group(1)
        
        venues_str = "札幌|函館|福島|新潟|東京|中山|中京|京都|阪神|小倉"
        match_meta = re.search(rf'(\d+)回({venues_str})(\\d+)日', full_text)
        if match_meta:
            kai = f"{int(match_meta.group(1)):02}"
            venue_text = match_meta.group(2)
            day = f"{int(match_meta.group(3)):02}"
            
        match_race = re.search(r'(\d+)レース', full_text)
        if match_race: r_num = f"{int(match_race.group(1)):02}"
        
        place_map = {
            "札幌": "01", "函館": "02", "福島": "03", "新潟": "04", "東京": "05",
            "中山": "06", "中京": "07", "京都": "08", "阪神": "09", "小倉": "10"
        }
        p_code = place_map.get(venue_text, "00")
        year = date_text[:4] if date_text else "2025"
        
        race_id = f"{year}{p_code}{kai}{day}{r_num}"
        
        # SKIP Check
        if existing_race_ids and race_id in existing_race_ids:
            return None

        # Basic Race Info
        race_name = soup.select_one(".race_name").text.strip() if soup.select_one(".race_name") else ""
        header_text = soup.select_one("div.header_line").text if soup.select_one("div.header_line") else full_text
        
        # Course/Dist
        course_type = ""; distance = ""
        dt_match = re.search(r'(芝|ダ|ダート|障害)[^0-9]*(\d+)', header_text)
        if dt_match:
            c = dt_match.group(1)
            course_type = '芝' if '芝' in c else ('ダート' if 'ダ' in c else '障害')
            distance = dt_match.group(2)
            
        # Weather/Condition
        weather = ""; condition = ""; rotation = ""
        w_match = re.search(r'天候\s*[:：]\s*(\S+)', soup.text)
        if w_match: weather = w_match.group(1)
        
        c_match = re.search(r'(?:芝|ダート)\s*[:：]\s*(\S+)', soup.text)
        if c_match: condition = c_match.group(1)
        
        if "右" in header_text: rotation = "右"
        elif "左" in header_text: rotation = "左"
        elif "直" in header_text: rotation = "直線"

        # --- Parse Result Table ---
        tables = soup.find_all('table')
        target_table = None
        for t in tables:
            if "着順" in t.text and "馬名" in t.text:
                target_table = t
                break
        
        if not target_table: return None
        
        rows = target_table.find_all('tr')
        race_scraper = RaceScraper()
        
        data_list = []
        
        # Pre-convert date for filtering
        d_obj = pd.to_datetime(date_text, format='%Y年%m月%d日') if date_text else datetime.now()
        
        print(f"    Fetching details for {len(rows)-1} horses...")
        
        for row in rows:
            if row.find('th'): continue # Skip header
            cells = row.find_all('td')
            if not cells: continue
            
            # Basic info
            rank = cells[0].text.strip()
            waku = ""
            if cells[1].find('img'): 
                alt = cells[1].find('img').get('alt', '')
                m = re.search(r'枠(\d+)', alt)
                waku = m.group(1) if m else alt
            umaban = cells[2].text.strip()
            horse_name_elem = cells[3].find('a')
            horse_name = cells[3].text.strip()
            horse_id = ""
            if horse_name_elem and 'href' in horse_name_elem.attrs:
                hm = re.search(r'/horse/(\d+)', horse_name_elem['href'])
                if hm: horse_id = hm.group(1)
            
            jockey = cells[6].text.strip()
            time_val = cells[7].text.strip()
            # ... other basic fields
            
            # --- Rich Fetching (History & Pedigree) ---
            blood_data = {"father": "", "mother": "", "bms": ""}
            past_data = {}
            
            if horse_id:
                # 1. Pedigree
                blood_data = race_scraper.get_horse_profile(horse_id)
                
                # 2. History
                df_past = race_scraper.get_past_races(horse_id, d_obj, n_samples=5)
                
                # Flatten History
                for i in range(5):
                    prefix = f"past_{i+1}"
                    if i < len(df_past):
                        r = df_past.iloc[i]
                        past_data[f"{prefix}_date"] = r.get('date', '')
                        past_data[f"{prefix}_rank"] = r.get('rank', '')
                        past_data[f"{prefix}_time"] = r.get('time', '')
                        past_data[f"{prefix}_run_style"] = r.get('run_style', '')
                        past_data[f"{prefix}_race_name"] = r.get('race_name', '')
                        past_data[f"{prefix}_last_3f"] = r.get('last_3f', '')
                        past_data[f"{prefix}_horse_weight"] = r.get('horse_weight', '')
                        past_data[f"{prefix}_jockey"] = r.get('jockey', '')
                        past_data[f"{prefix}_condition"] = r.get('condition', '')
                        past_data[f"{prefix}_odds"] = r.get('odds', '')
                        past_data[f"{prefix}_weather"] = r.get('weather', '')
                        past_data[f"{prefix}_distance"] = r.get('distance', '')
                        past_data[f"{prefix}_course_type"] = r.get('course_type', '')
                    else:
                        # Fill Empty
                        for col in ['date','rank','time','run_style','race_name','last_3f','horse_weight','jockey','condition','odds','weather','distance','course_type']:
                            past_data[f"{prefix}_{col}"] = ""

                # Be polite between horses
                time.sleep(0.5)
            
            # Combine
            row_dict = {
                "日付": date_text, "会場": venue_text, "レース番号": f"{r_num}R", "レース名": race_name, "重賞": "",
                "コースタイプ": course_type, "距離": distance, "回り": rotation, "天候": weather, "馬場状態": condition,
                "着順": rank, "枠": waku, "馬番": umaban, "馬名": horse_name, 
                "性齢": cells[4].text.strip(), "斤量": cells[5].text.strip(), "騎手": jockey, 
                "タイム": time_val, "着差": cells[8].text.strip(), "人気": cells[13].text.strip(), 
                "単勝オッズ": cells[14].text.strip() if len(cells)>14 else "0.0", 
                "後3F": cells[10].text.strip(), "厩舎": cells[12].text.strip(), 
                "馬体重(増減)": cells[11].text.strip(),
                "race_id": race_id, "horse_id": horse_id,
                **blood_data,
                **past_data
            }
            data_list.append(row_dict)
            
        df = pd.DataFrame(data_list)
        
        # Enforce User-Specified Column Order
        ordered_columns = [
            "日付", "会場", "レース番号", "レース名", "重賞", "コースタイプ", "距離", "回り", "天候", "馬場状態",
            "着順", "枠", "馬番", "馬名", "性齢", "斤量", "騎手", "タイム", "着差", "人気", "単勝オッズ",
            "後3F", "厩舎", "馬体重(増減)", "race_id", "horse_id"
        ]
        # rich data columns
        for i in range(1, 6):
            p = f"past_{i}"
            ordered_columns.extend([
                f"{p}_date", f"{p}_rank", f"{p}_time", f"{p}_run_style", f"{p}_race_name",
                f"{p}_last_3f", f"{p}_horse_weight", f"{p}_jockey", f"{p}_condition",
                f"{p}_odds", f"{p}_weather", f"{p}_distance", f"{p}_course_type"
            ])
        ordered_columns.extend(["father", "mother", "bms"])
        
        # Add missing cols with empty string, remove extras (if any/optional)
        # reindex handles this safely
        df_ordered = df.reindex(columns=ordered_columns, fill_value="")
        
        return df_ordered

    except Exception as e:
        print(f"Error scraping race {url}: {e}")
        return None

# --- Year/Month Iteration Logic (Scrape Year Rich) ---
def scrape_jra_year_rich(year_str, start_date=None, end_date=None, save_callback=None, existing_race_ids=None):
    # Parameter Map for Monthly Results
    JRA_MONTH_PARAMS = {
        "2026": { "01": "E4", "02": "B2", "03": "80", "04": "4E", "05": "1C", "06": "EA", "07": "B8", "08": "86", "09": "54", "10": "22", "11": "F0", "12": "BE" },
        "2025": { "01": "3F", "02": "0D", "03": "DB", "04": "A9", "05": "77", "06": "45", "07": "13", "08": "E1", "09": "AF", "10": "1E", "11": "EC", "12": "D3" },
        "2024": { "01": "B3", "02": "81", "03": "4F", "04": "1D", "05": "EB", "06": "B9", "07": "87", "08": "55", "09": "23", "10": "92", "11": "60", "12": "2E" },
        "2023": { "01": "27", "02": "F5", "03": "C3", "04": "91", "05": "5F", "06": "2D", "07": "FB", "08": "C9", "09": "97", "10": "06", "11": "D4", "12": "A2" },
        "2022": { "01": "9B", "02": "69", "03": "37", "04": "05", "05": "D3", "06": "A1", "07": "6F", "08": "3D", "09": "0B", "10": "7A", "11": "48", "12": "16" },
        "2021": { "01": "0F", "02": "DD", "03": "AB", "04": "79", "05": "47", "06": "15", "07": "E3", "08": "B1", "09": "7F", "10": "EE", "11": "BC", "12": "8A" },
        "2020": { "01": "83", "02": "51", "03": "1F", "04": "ED", "05": "BB", "06": "89", "07": "57", "08": "25", "09": "F3", "10": "62", "11": "30", "12": "FE" }
    }
    
    if year_str not in JRA_MONTH_PARAMS:
        print(f"年度 {year_str} はサポートされていません。")
        return

    params = JRA_MONTH_PARAMS[year_str]
    base_url = "https://www.jra.go.jp/JRADB/accessS.html"

    # Determine months to iterate
    start_m = 1
    end_m = 12

    if start_date:
        start_m = start_date.month
    if end_date:
        end_m = end_date.month

    # Cap at Today
    from datetime import date
    today = date.today()

    if end_date:
        actual_end_date = min(end_date, today)
    else:
        actual_end_date = today

    print(f"=== JRA 一括スクレイピング開始 {year_str} (リッチモード) ===")
    print(f"期間: {start_date or 'Start'} - {actual_end_date}")
    
    if int(year_str) == today.year:
        end_m = min(end_m, today.month)

    total_processed = 0

    for m in range(start_m, end_m + 1):
        month = f"{m:02}"
        if month not in params:
            continue

        suffix = params[month]
        try:
            ym = int(year_str + month)
            prefix = "pw01skl00" if ym >= 202512 else "pw01skl10"
        except:
            prefix = "pw01skl10"

        cname = f"{prefix}{year_str}{month}/{suffix}"

        print(f"\n📅 {year_str}/{month} を取得中...")

        try:
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            }
            response = requests.post(base_url, data={"cname": cname}, headers=headers, timeout=15)
            response.encoding = 'cp932'

            if response.status_code != 200:
                print(f"❌ {cname} の取得に失敗しました (Status {response.status_code})")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')

            race_cnames = []
            links = soup.find_all('a')
            for link in links:
                onclick = link.get('onclick', '')
                match = re.search(r"doAction\('[^']+',\s*'([^']+)'\)", onclick)
                if match:
                    c = match.group(1)
                    if c.startswith('pw01srl'):
                        race_cnames.append(c)

            race_cnames = sorted(list(set(race_cnames)))
            print(f"  {len(race_cnames)} 開催日が見つかりました")

            for day_cname in tqdm(race_cnames, desc=f"  {year_str}/{month}", leave=False):
                resp_day = requests.post(base_url, data={"cname": day_cname}, headers=headers, timeout=15)
                resp_day.encoding = 'cp932'
                soup_day = BeautifulSoup(resp_day.text, 'html.parser')

                race_list_items = []
                all_anchors = soup_day.find_all('a')
                for a in all_anchors:
                    onclick = a.get('onclick', '')
                    match_sde = re.search(r"doAction\s*\(\s*['\"][^'\"]+['\"]\s*,\s*['\"](pw01sde[^'\"]+)['\"]\s*\)", onclick)
                    href = a.get('href', '')

                    final_url = ""
                    if match_sde:
                        final_url = f"{base_url}?CNAME={match_sde.group(1)}"
                    elif 'pw01sde' in href:
                         if 'CNAME=' in href:
                             final_url = urllib.parse.urljoin(base_url, href)
                         else:
                             final_url = urllib.parse.urljoin(base_url, href)

                    if final_url:
                        race_list_items.append(final_url)

                unique_races = sorted(list(set(race_list_items)))

                daily_data = []

                for r_link in unique_races:
                    # Fetch with Rich Scraper
                    df = scrape_race_rich(r_link, existing_race_ids=existing_race_ids)

                    if df is not None and not df.empty:
                        daily_data.append(df)
                        total_processed += 1
                    
                    # Rate limiting
                    time.sleep(1.0) 
                
                # Save daily batch
                if daily_data and save_callback:
                    df_day = pd.concat(daily_data, ignore_index=True)
                    save_callback(df_day) 

        except Exception as e:
            print(f"❌ {month}月の処理中にエラーが発生しました: {e}")
            
    print("完了しました。")


In [ ]:
# NAR スクレイピングロジック
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import date, timedelta
import calendar
import time
import os

def run_nar_scraping(year, start_month=1, end_month=12, save_dir='data/raw'):
    from tqdm.auto import tqdm
    
    s_date = date(int(year), int(start_month), 1)
    last_day_e = calendar.monthrange(int(year), int(end_month))[1]
    e_date = date(int(year), int(end_month), last_day_e)
    
    today = date.today()
    if e_date > today: e_date = today
    
    print(f'NARデータを {s_date} から {e_date} まで取得します...')
    print(f'保存先: {os.path.join(save_dir, "database_nar.csv")}')
    
    # 月ごとにループ
    for m in range(int(start_month), int(end_month) + 1):
        # その月の日付範囲を決定
        m_start = date(int(year), m, 1)
        m_last = calendar.monthrange(int(year), m)[1]
        m_end = date(int(year), m, m_last)
        
        # 範囲外ならスキップ
        if m_end < s_date or m_start > e_date:
            continue
            
        # 実際の開始・終了（クランプ）
        curr_s = max(m_start, s_date)
        curr_e = min(m_end, e_date)
        
        if curr_s > curr_e: continue
        
        # 日付リスト作成
        days = []
        c = curr_s
        while c <= curr_e:
            days.append(c)
            c += timedelta(days=1)
            
        print(f'\n📅 {year}/{m:02} を取得中...')
        print(f'  {len(days)} 日分の日付対象')
        
        for d in tqdm(days, desc=f'  {year}/{m:02}'):
            d_str = d.strftime('%Y%m%d')
            url = f'https://nar.netkeiba.com/top/race_list_sub.html?kaisai_date={d_str}'
            
            daily_data = [] # 1日分のデータを貯める
            
            try:
                 time.sleep(0.5)
                 headers = {'User-Agent': 'Mozilla/5.0'}
                 resp = requests.get(url, headers=headers)
                 resp.encoding = 'EUC-JP'
                 soup = BeautifulSoup(resp.text, 'html.parser')
                 links = soup.select('a[href*="race/result.html"]')
                 
                 if links:
                     # print(f'  {d}: {len(links)} レース') # ログ過多防止のためコメントアウト
                     for link in links:
                         href = link.get('href')
                         if href.startswith('../'):
                             full_url = f'https://nar.netkeiba.com/{href.replace("../", "")}'
                         elif href.startswith('http'):
                             full_url = href
                         else:
                             full_url = f'https://nar.netkeiba.com{href}'
                         
                         try:
                             df = scrape_race_rich(full_url, existing_race_ids=None)
                             if df is not None and not df.empty:
                                 daily_data.append(df)
                             time.sleep(1)
                         except Exception as e_race:
                             pass # エラーは無視して次へ
                 
                 # 1日分のループ終了後、まとめて保存
                 if daily_data:
                     os.makedirs(save_dir, exist_ok=True)
                     csv_file = os.path.join(save_dir, 'database_nar.csv')
                     try:
                         df_day = pd.concat(daily_data, ignore_index=True)
                         
                         if not os.path.exists(csv_file):
                             df_day.to_csv(csv_file, index=False)
                         else:
                             existing_cols = pd.read_csv(csv_file, nrows=0).columns.tolist()
                             df_aligned = df_day.reindex(columns=existing_cols)
                             df_aligned.to_csv(csv_file, mode='a', header=False, index=False)
                     except Exception as e_save:
                          print(f"  保存エラー ({d}): {e_save}")
            
            except Exception as e_day:
                print(f'  日付処理エラー {d}: {e_day}')
    
    print('完了しました。')


In [ ]:
# 設定 (ここを変更してください)
YEAR = 2024          # 対象年度
START_MONTH = 1      # 開始月
END_MONTH = 12       # 終了月
SAVE_DIR = '/content/drive/MyDrive/dai-keiba/data/raw' # 保存先フォルダ

# 実行ブロック
if YEAR:
    # ディレクトリ作成
    os.makedirs(SAVE_DIR, exist_ok=True)
    run_nar_scraping(YEAR, START_MONTH, END_MONTH, save_dir=SAVE_DIR)


In [ ]:
# データ整理・重複削除・カラム順序保証 (NAR)
import pandas as pd
import os

csv_path = os.path.join(SAVE_DIR, 'database_nar.csv')
if os.path.exists(csv_path):
    print('データの整理を行っています...')
    try:
        # 全カラムを文字列として読み込み（型ずれ防止）
        df_final = pd.read_csv(csv_path, dtype=str)
        before_len = len(df_final)
        
        # 重複排除: race_idとhorse_idが同じなら、後勝ち（最新）を採用
        if 'race_id' in df_final.columns and 'horse_id' in df_final.columns:
            df_final.drop_duplicates(subset=['race_id', 'horse_id'], keep='last', inplace=True)
        
        after_len = len(df_final)
        print(f'重複削除: {before_len} -> {after_len} ({before_len - after_len}件削除)')
        
        df_final.to_csv(csv_path, index=False)
        print('完了: データの整合性を確認し保存しました。')
    except Exception as e:
        print(f'データ整理中にエラーが発生しました: {e}')
